In [8]:
import numpy as np
import matplotlib.pyplot as plt


In [9]:
# Discretization parameters
dx = 0.01 # space step
t_end = 5 # end of simulation time 
gamma = 1.4
c = 0.8

In [10]:
 
x_start = 0
x_end = 10+dx

_x = np.arange(x_start, x_end, dx)

delta_x_ = 1 # Interval of the "disturbance"


### Contact Discontinuity

In [11]:
def top_hat(x, x_start, width):
    # Top hat parametrization
    # 2     ___
    #      |   |
    # 1 ___|   |___
    #   0  1   2  3

    if x_start <= x <= x_start + width:
        return 2
    else:
        return 1

In [12]:
# Contact discontinuity through LFx2
# Velocity is zero
u0 = np.zeros_like(_x)
# Pressure is 1
p0 = np.ones_like(_x)
# Top hat initial condition for density
rho0 = np.array([top_hat(x, 1, 1) for x in _x])


U0 = np.array([rho0, u0, p0]).T

In [6]:
from lfx2 import LFx2
U_prop, dt_arr = LFx2(U0, dx, c, t_end, gamma)

ValueError: could not broadcast input array from shape (999,3) into shape (1001,3)

In [ ]:
#Unpack the solution, U_prop is a 3D array [n_step, n_sample, n_var]
rho = U_prop[:,:,0]

u = U_prop[:,:,1]

p = U_prop[:,:,2]


### Expansion

### Compression

### Shock